In [ ]:
import pandas
import json
import os

In [ ]:
main_df = pandas.read_csv('mitocells.csv')

In [ ]:
to_concat = ['granularity','shape','texture5','texture3','texture10']
cols2D=['max_projection_x','max_projection_y','max_projection_z','mean_projection_x','mean_projection_y','mean_projection_z','center_slice',]
cell_stages=["M0","M1M2","M3","M4M5","M6M7_complete"]

In [ ]:
for eachstage in cell_stages:
    stage_df = main_df.query(f"cell_stage == '{eachstage}'")
    for eachfeature in to_concat:
        for eachproj in cols2D:
            try:
                print(f"{eachfeature}/mitocells{eachstage}{eachproj}.csv")
                feature_df = pandas.read_csv(f"{eachfeature}/mitocells{eachstage}{eachproj}.csv")
            except FileNotFoundError:
                with open(f"{eachfeature}/texture_5_{eachstage}_{eachproj}.json","r") as jsonfile:
                    feature_df = pandas.DataFrame(json.load(jsonfile))
            feature_columns_lower = [x.lower() for x in feature_df.columns]
            feature_df_columns = [x for x in feature_columns_lower if eachfeature[:-1] in x]
            if eachfeature == 'shape':
                feature_df_columns = [x for x in feature_df_columns if 'shape_mode' not in x]
            if eachstage != "M0":
                if "angle" in feature_df.columns:
                    feature_df = feature_df.query("angle != angle") #remove orginal rows
                    feature_df = feature_df.reset_index(drop=True)
                stage_df = stage_df.reset_index(drop=True)
            feature_columns_to_use = [x for x in feature_df.columns if x.lower() in feature_df_columns]
            stage_df = stage_df.join(feature_df[feature_columns_to_use])
    stage_df.to_csv(f"cells_with_features_stage_{eachstage}.csv",index=False)
                

In [ ]:
cells_with_features_list=[]
for eachstage in cell_stages:
    cells_with_features_list.append(pandas.read_csv(f"cells_with_features_stage_{eachstage}.csv"))
cells_df = pandas.concat(cells_with_features_list)
cells_df.to_csv('all_cells_with_features.csv',index=False)